In [1]:
import pandas as pd
import seaborn as sns

import ipywidgets as widgets
from ipywidgets import interact

import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("data.csv", parse_dates=True, index_col=0)
data.head()

,flow,acc_precip,mean_radiation,mean_temp
time,,,,
2023-11-07 00:00:00+00:00,2537.061000,4.6,937.0,8.366667
2023-11-08 00:00:00+00:00,1988.173274,1.8,0.0,8.285714
2023-11-09 00:00:00+00:00,3005.526184,5.8,337.0,8.530000
2023-11-10 00:00:00+00:00,3490.331014,4.2,689.0,7.275000
2023-11-14 00:00:00+00:00,3194.873297,2.8,266.7,5.277778


In [3]:
features = ["mean_radiation", "mean_temp", "acc_precip"]
feature_selector = widgets.SelectMultiple(
    options=features,
    value=features[1:],
    description="Variables:",
)

freqs = ["D", "W", "2W", "ME"]
freq_selector = widgets.RadioButtons(
    options=freqs,
    description="Temporal resolution:",
    disabled=False,
    orientation="horizontal",
)

display(feature_selector)

SelectMultiple(description='Variables:', index=(1, 2), options=('mean_radiation', 'mean_temp', 'acc_precip'), …

In [4]:
def select_and_resample(freq, features):
    resample_dict = {
        "flow": "sum",
        "mean_radiation": "mean",
        "mean_temp": "mean",
        "acc_precip": "sum",
    }
    vars = ["flow"] + features
    return (
        data.loc[:, vars]
        .resample(freq)
        .agg({k: v for k, v in resample_dict.items() if k in vars})
    )


hue_options = set(features).difference(set(feature_selector.value))
hue_picker = widgets.ToggleButtons(
    options=hue_options,
    description="Hue:",
)


@interact(freq=freq_selector, hue=hue_picker)
def make_pairplot(freq, hue):
    if hue is not None:
        additional_features = list(feature_selector.value) + [hue]
    else:
        additional_features = list(feature_selector.value)
    resampled_data = select_and_resample(freq, additional_features)
    sns.pairplot(resampled_data, hue=hue)

interactive(children=(RadioButtons(description='Temporal resolution:', options=('D', 'W', '2W', 'ME'), orienta…

In [5]:
features = list(feature_selector.value) + [hue_picker.value]
selected_data = select_and_resample(freq_selector.value, features)
selected_data.describe()

,flow,mean_radiation,mean_temp,acc_precip
count,469.000000,441.000000,441.000000,469.000000
mean,1466.120824,2316.382313,8.011601,2.470576
std,810.954606,2319.086878,5.928460,5.290488
min,0.000000,0.000000,-7.421429,0.000000
25%,989.205031,389.000000,3.708333,0.000000
50%,1268.577611,1307.000000,7.325000,0.100000
75%,1768.471276,3981.000000,13.295833,2.600000
max,5354.692318,8607.000000,20.887500,45.900000


In [6]:
datplot = selected_data.loc[selected_data["acc_precip"] < 40, :]

temp_selector = widgets.FloatRangeSlider(min=-7, max=23, readout_format=".1f")


@interact()
def plot_reg_by_temperature(temp_cutoff=temp_selector):
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    sns.scatterplot(datplot, x="acc_precip", y="flow", ax=ax, alpha=0.3)
    sns.regplot(
        datplot.loc[datplot["mean_temp"].between(*temp_cutoff), :],
        x="acc_precip",
        y="flow",
        ax=ax,
        scatter_kws=dict(alpha=0),
    )

interactive(children=(FloatRangeSlider(value=(0.5, 15.5), description='temp_cutoff', max=23.0, min=-7.0, reado…